In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
torch.accelerator.current_accelerator().type

'mps'

"mps"指的是 Metal Performance Shaders（MPS），这是苹果提供的一个高效计算框架，专为在Apple设备上的GPU（图形处理单元）上进行高性能计算任务而设计。MPS允许开发者在支持Metal的设备（如Mac、iPhone、iPad）上利用GPU来加速计算，比如机器学习、图像处理和数据计算等操作

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0222, -0.0007,  0.0202,  0.0700, -0.0046, -0.0010,  0.0437, -0.0833,
          0.0369,  0.0201]], device='mps:0', grad_fn=<LinearBackward0>)
Predicted class: tensor([3], device='mps:0')


In [8]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0234,  0.0250,  0.0190,  ..., -0.0257, -0.0173, -0.0096],
        [ 0.0314,  0.0186, -0.0078,  ...,  0.0323, -0.0152, -0.0073]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0110, -0.0327], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0364,  0.0230,  0.0267,  ...,  0.0299,  0.0325, -0.0364],
        [-0.0067, -0.0353,  0.0396,  ..., -0.0026, -0.0149, -0.0208]],
       device='mps:0', grad_fn=<Slice